# Patent Assignment Daily
Contains daily patent assignment text for 10/18/2016

In [7]:
import pandas as pd
import numpy as np

In [8]:
assignments = pd.read_csv('patent_assignment.csv', index_col= 0)

In [9]:
assignments.head()

,last-update-date,patent-assignees,patent-assignors,patent-countries,patent-dates,patent-kinds,patent-numbers,recorded-date,title
0,20161018,FASTCASE,"WALTERS, EDWARD J. III|ROSENTHAL, PHILIP J.",US|US,20001108|20161018,X0|B1,09707911|9471672,20010320,Relevance sorting for database searches
1,20161018,ANABASIS SRL,"LAMBIASE, ALESSANDRO",US|US,20010726|20161018,X0|B1,09890088|9468665,20010720,METHOD OF TREATING INTRAOCCULAR TISSUE PATHOLO...
2,20161018,QUALCOMM INCORPORATED,"WALTON, J. RODNEY|KETCHUM, JOHN W.",US|US|US,20031201|20050602|20161018,X0|A1|B2,10725904|20050120097|9473269,20031201,METHOD AND APPARATUS FOR PROVIDING AN EFFICIEN...
3,20161018,INTERNATIONAL BUSINESS MACHINES CORPORATION,"MORARIU, JANIS A.|STAPEL, STEVEN W.|STRAACH, J...",US|US|US,20040622|20051222|20161018,X0|A1|B2,10873346|20050282136|9472114,20040903,"COMPUTER-IMPLEMENTED METHOD, SYSTEM AND PROGRA..."
4,20161018,INTERNATIONAL BUSINESS MACHINES CORPORATION,"LI, XIN|ROBERTS, GREGORY WAYNE",US|US|US,20041019|20060420|20161018,X0|A1|B2,10967958|20060085754|9471332,20050208,Selecting graphical component types at runtime


In [10]:
titles_in_data=assignments['title']

In [11]:
chars = list(set(titles_in_data)) 

In [12]:
data_size, vocab_size = len(titles_in_data), len(chars)
print ("Patent data has chars", data_size)
print ("Patent data has unique chars", vocab_size)

Patent data has chars 7530
Patent data has unique chars 6441


## Topic Modeling

In [13]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import nltk
import gensim

### tokenize

The first thing we have to do is **tokenize** our words. A naive way to do this would be to split our string based on spaces (e.g. str.split(" ")), which is sometimes OK but has many edge cases (alternative punctuation marks like —, for example) and will fail to work as expected for larger problems.

nltk comes with a built-in word tokenizer that we can take advantage of.

In [14]:
titles = assignments['title']
title_tokens = [nltk.word_tokenize(title) for title in\
                    np.concatenate(titles.map(str).map(str.title).map(lambda s: s.split("|")))]

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hassan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
title_tokens = [title for title in title_tokens if len(title_tokens) > 0]

In [17]:
len(title_tokens)

370460

In [18]:
title_tokens[:3]

[['Relevance', 'Sorting', 'For', 'Database', 'Searches'],
 ['Method',
  'Of',
  'Treating',
  'Intraoccular',
  'Tissue',
  'Pathologies',
  'With',
  'Nerve',
  'Growth',
  'Factor',
  '.'],
 ['Method',
  'And',
  'Apparatus',
  'For',
  'Providing',
  'An',
  'Efficient',
  'Control',
  'Channel',
  'Structure',
  'In',
  'A',
  'Wireless',
  'Communication',
  'System']]

In [19]:
print(title_tokens[2])

['Method', 'And', 'Apparatus', 'For', 'Providing', 'An', 'Efficient', 'Control', 'Channel', 'Structure', 'In', 'A', 'Wireless', 'Communication', 'System']


### stem

Next, we will stem our words. Stemming is a procedure in natural language processing where we chop off everything except for the root of a word. So for example, the words go, going, and gone will all map to the same root—go.

This is a good thing to do, particularly given the small size of our documents, because it increases the accuracy of classifications—more things end up being the same.

nltk comes with several stemmers installed, we'll use the PorterStemmer

In [20]:
stemmer = nltk.stem.PorterStemmer()
titles_stemmed = [[stemmer.stem(token) for token in tokens] for tokens in title_tokens]

In [21]:
titles_stemmed[:3]

[['relev', 'sort', 'for', 'databas', 'search'],
 ['method',
  'Of',
  'treat',
  'intraoccular',
  'tissu',
  'patholog',
  'with',
  'nerv',
  'growth',
  'factor',
  '.'],
 ['method',
  'and',
  'apparatu',
  'for',
  'provid',
  'An',
  'effici',
  'control',
  'channel',
  'structur',
  'In',
  'A',
  'wireless',
  'commun',
  'system']]

In [22]:
print(titles_stemmed[2])

['method', 'and', 'apparatu', 'for', 'provid', 'An', 'effici', 'control', 'channel', 'structur', 'In', 'A', 'wireless', 'commun', 'system']


If we examine a list of words, however, we see that the most common English-language words dominate:

In [23]:
print(pd.Series(np.concatenate(titles_stemmed)).value_counts())

and                             170856
for                             145224
method                          137910
A                               101633
Of                               91017
system                           75843
devic                            65015
with                             46433
In                               39461
,                                37235
apparatu                         31147
circuit                          29046
memori                           28675
semiconductor                    26290
use                              24697
To                               23959
data                             22710
An                               22235
control                          20957
the                              20244
have                             17662
integr                           16994
process                          16233
structur                         16052
network                          14185
form                     

These words carry no meaning and aren't very interesting.For example, we can see "," has occurance of  37235. <br>

They're known as **stopwords** in NLP, and we're going to once again use  nltk builtins to remove them from consideration.

### stopwords

In [24]:
from nltk.corpus import stopwords

In [25]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hassan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
english_stopwords = set([word.title() for word in stopwords.words("english")])

In [27]:
stemmed_title_words = [[word for word in title if word not in english_stopwords] for title in titles_stemmed]

In [28]:
stemmed_title_words[:3]

[['relev', 'sort', 'for', 'databas', 'search'],
 ['method',
  'treat',
  'intraoccular',
  'tissu',
  'patholog',
  'with',
  'nerv',
  'growth',
  'factor',
  '.'],
 ['method',
  'and',
  'apparatu',
  'for',
  'provid',
  'effici',
  'control',
  'channel',
  'structur',
  'wireless',
  'commun',
  'system']]

In [29]:
print(stemmed_title_words[2])

['method', 'and', 'apparatu', 'for', 'provid', 'effici', 'control', 'channel', 'structur', 'wireless', 'commun', 'system']


In [30]:
word_counts = pd.Series(np.concatenate(stemmed_title_words)).value_counts()
singular_words = set(word_counts[pd.Series(np.concatenate(stemmed_title_words)).value_counts() == 1].index)

In [31]:
stemmed_title_common_words = [[word for word in title if word not in singular_words] for title in stemmed_title_words]

In [32]:
stemmed_title_common_words[:3]

[['relev', 'sort', 'for', 'databas', 'search'],
 ['method',
  'treat',
  'intraoccular',
  'tissu',
  'patholog',
  'with',
  'nerv',
  'growth',
  'factor',
  '.'],
 ['method',
  'and',
  'apparatu',
  'for',
  'provid',
  'effici',
  'control',
  'channel',
  'structur',
  'wireless',
  'commun',
  'system']]

In [33]:
print(stemmed_title_common_words[2])

['method', 'and', 'apparatu', 'for', 'provid', 'effici', 'control', 'channel', 'structur', 'wireless', 'commun', 'system']


Next, let's consider the opposite problem: words that occur to infrequently to be useful. Words that only ever appear once, for example, don't carry any information. Remember, we're going to split all of our patent titles into some small number of classes; just as in any other dataset, a data point which is only populated once isn't interesting, and can be safely dropped.

In fact, we could probably drop a lot of words from consideration, not just ones appearing once but ones appearing tens or even hundreds of times. This would speed up our algorithms and won't significantly impact our results.

After a certain point words do start to matter, however; figuring out where that point is is up to you.

In our case we'll just be lazy and cut off at words that appear only once, and leave words appearing twice or more intact.

In [34]:
non_empty_indices = [i for i in range(len(stemmed_title_common_words)) if len(stemmed_title_common_words[i]) > 0]

In [35]:
non_empty_indices[5000]

5003

Notice that discarding words from our set has resulted in a handful of empty titles. Apparently a few patents have nothing but unique words!

In [36]:
stemmed_title_common_words_nonnull = np.asarray(stemmed_title_common_words)[non_empty_indices]

In [37]:
classifiable_titles = np.asarray(title_tokens)[non_empty_indices]

In [38]:
classifiable_titles[:5]

array([list(['Relevance', 'Sorting', 'For', 'Database', 'Searches']),
       list(['Method', 'Of', 'Treating', 'Intraoccular', 'Tissue', 'Pathologies', 'With', 'Nerve', 'Growth', 'Factor', '.']),
       list(['Method', 'And', 'Apparatus', 'For', 'Providing', 'An', 'Efficient', 'Control', 'Channel', 'Structure', 'In', 'A', 'Wireless', 'Communication', 'System']),
       list(['Computer-Implemented', 'Method', ',', 'System', 'And', 'Program', 'Product', 'For', 'Providing', 'An', 'Educational', 'Program']),
       list(['Selecting', 'Graphical', 'Component', 'Types', 'At', 'Runtime'])],
      dtype=object)

With our titles adequately processed, now we switch over to gensim. The first thing we have to do is build a dictionary of words, which associates each word [stem] with a particular index number:

In [39]:
dictionary = gensim.corpora.Dictionary(stemmed_title_common_words_nonnull)

In [40]:
str(dictionary.token2id)[:1000]

"{'databas': 0, 'for': 1, 'relev': 2, 'search': 3, 'sort': 4, '.': 5, 'factor': 6, 'growth': 7, 'intraoccular': 8, 'method': 9, 'nerv': 10, 'patholog': 11, 'tissu': 12, 'treat': 13, 'with': 14, 'and': 15, 'apparatu': 16, 'channel': 17, 'commun': 18, 'control': 19, 'effici': 20, 'provid': 21, 'structur': 22, 'system': 23, 'wireless': 24, ',': 25, 'computer-impl': 26, 'educ': 27, 'product': 28, 'program': 29, 'compon': 30, 'graphic': 31, 'runtim': 32, 'select': 33, 'type': 34, 'aggreg': 35, 'chromatographi': 36, 'high': 37, 'hydroxyapatit': 38, 'molecular': 39, 'remov': 40, 'use': 41, 'weight': 42, 'aid': 43, 'differ': 44, 'further': 45, 'protocol': 46, 'station': 47, 'the': 48, 'transpond': 49, 'convert': 50, 'input': 51, 'languag': 52, 'output': 53, 'phonet': 54, 'written': 55, 'dataset': 56, 'from': 57, 'link': 58, 'methodolog': 59, 'multi-mod': 60, 'pattern': 61, 'charact': 62, 'digit': 63, 'media': 64, 'person': 65, 'replac': 66, 'airway': 67, 'detect': 68, 'instabl': 69, 'align': 7

Why are we doing this? Because shortly we're going to throw our corpus into a tf-idf algorithm. TF-IDF is an algorithm in information retrieval which converts a list of word "vectors" to a scaled Euclidian normal vector. It turns a count of the number of each word in our document into a unit vector in N-dimensional space, where N is, believe it or not, the number of individual words that we have in our dictionary (above).

That means that, in this case, we have a "dataset" matrix with hundreds of thousands of columns in it!

The beauty of TD-IDF is that it scales the words according to how frequent or rare they are. Words that appear a lot in your text but also appear a lot in the rest of the corpus are weighed less heavily than words that appear a lot in your text but more rarely outside of it.

Thus we first use gensim to convert our words to word incidence vectors...

In [41]:
corpus = [dictionary.doc2bow(text) for text in stemmed_title_common_words_nonnull]

In [42]:
stemmed_title_common_words_nonnull[0], corpus[0]

(['relev', 'sort', 'for', 'databas', 'search'],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)])

In [43]:
print(stemmed_title_common_words_nonnull[2], corpus[2])

['method', 'and', 'apparatu', 'for', 'provid', 'effici', 'control', 'channel', 'structur', 'wireless', 'commun', 'system'] [(1, 1), (9, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]


In [44]:
print(stemmed_title_common_words_nonnull[100], corpus[100])

['passeng', 'transport', 'system', 'and', 'method', 'for', 'obtain', 'ticket', 'such', 'system'] [(1, 1), (9, 1), (15, 1), (23, 2), (300, 1), (325, 1), (326, 1), (327, 1), (328, 1)]


..then run TfidfModel from gensim on them to turn them into our word vectors!

## Tfidf Model

In [45]:
from gensim.models import TfidfModel

In [46]:
tfidf = TfidfModel(corpus)

Note that gensim doesn't follow the scikit access pattern, if you are familiar with it. It instead (1) defers computations on individual entries until necessary and (2) provides access to data using bracket indexing notation ([]).

By contrast, scikit will run everything immediately by default, provides results using a .values_ attribute, and seperates model initialization from runtime (the latter doesn't occur until you fit() your model).

In [47]:
stemmed_title_common_words_nonnull[0], corpus[0], tfidf[corpus[0]]

(['relev', 'sort', 'for', 'databas', 'search'],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 0.4377475295381456),
  (1, 0.07562061510033051),
  (2, 0.5469009211535774),
  (3, 0.36992777626242423),
  (4, 0.6055670447985132)])

In [48]:
print(stemmed_title_common_words_nonnull[2], corpus[2], tfidf[corpus[2]])

['method', 'and', 'apparatu', 'for', 'provid', 'effici', 'control', 'channel', 'structur', 'wireless', 'commun', 'system'] [(1, 1), (9, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)] [(1, 0.08840592502566112), (9, 0.08992064121432716), (15, 0.07858769583335751), (16, 0.22360915542920912), (17, 0.42309521007730033), (18, 0.3118124008719702), (19, 0.26099879446104235), (20, 0.44892517091974105), (21, 0.3559213131631317), (22, 0.2847423513065349), (23, 0.14769708350864824), (24, 0.3904590429022807)]


With our words suitibly datified, we can now move on to fitting a model. Since our words are now, effectively, a very large dataset, it's possible to use any general purpose classifier to fit it.  For example,We could use a scipy KMeans clustering algorithm to arrive at its topics.

We'll instead use a model specifically adapted to natural language processing from the gensim built-ins, **LsiModel**.

## LSI Model

In [49]:
from gensim.models import LsiModel

Here's how we run it:

In [50]:
corpus_tfidf = tfidf[corpus]
lsi = LsiModel(tfidf[corpus], id2word=dictionary, num_topics=10)
corpus_lsi = lsi[corpus_tfidf]

Here's a printout of what words are important to our various topics. Notice that certain extremely common words, like semiconductor, appear in different positions in multiple classifiers. Also, note that this display is cut off at a certain number of displayed words; in reality the model considers far more than these (you can specify how many to display here, however, using the num_words parameter).

In [51]:
lsi.print_topics(10)

[(0,
  '0.381*"devic" + 0.350*"semiconductor" + 0.299*"method" + 0.281*"and" + 0.229*"for" + 0.228*"system" + 0.194*"," + 0.188*"memori" + 0.183*"circuit" + 0.153*"apparatu"'),
 (1,
  '-0.603*"semiconductor" + -0.353*"devic" + 0.327*"system" + 0.197*"apparatu" + 0.171*"for" + 0.167*"data" + 0.153*"and" + 0.152*"," + -0.146*"manufactur" + -0.132*"form"'),
 (2,
  '-0.719*"circuit" + -0.502*"integr" + 0.161*"system" + 0.121*"data" + 0.114*"semiconductor" + 0.112*"devic" + -0.112*"packag" + -0.100*"voltag" + 0.100*"commun" + 0.096*"network"'),
 (3,
  '0.673*"memori" + -0.342*"commun" + 0.282*"cell" + -0.196*"handl" + -0.172*"semiconductor" + -0.164*"inform" + 0.141*"non-volatil" + -0.124*"wireless" + -0.114*"devic" + 0.112*"form"'),
 (4,
  '-0.563*"," + -0.301*"imag" + -0.259*"apparatu" + 0.237*"handl" + 0.235*"system" + 0.210*"manag" + -0.188*"display" + 0.182*"power" + 0.163*"network" + 0.157*"inform"'),
 (5,
  '-0.575*"fiber" + -0.541*"optic" + -0.275*"cabl" + -0.273*"connector" + 0.181

Here are the scoring outputs for the first five documents:

In [52]:
for scores in corpus_lsi[:5]:
    print(scores)

[(0, 0.025505540264359014), (1, 0.027408435972474353), (2, 0.014314312172872854), (3, 0.0009359003513390955), (4, 0.010254277037139435), (5, 0.003205037218056633), (6, 0.007512579061520536), (7, 0.0007182907692681364), (8, 0.0012199950227529148), (9, -0.0043872199061536134)]
[(0, 0.0263524443272887), (1, 0.0071964231167035795), (2, -0.0016039044408344867), (3, 0.00456722852921116), (4, 0.0034914169307697406), (5, -0.01016869765142659), (6, -0.0021337653886918167), (7, -0.006471794961194971), (8, 0.0008302112498956094), (9, -0.01670786188977747)]
[(0, 0.2646618820986156), (1, 0.22642697512240811), (2, 0.10993306476517442), (3, -0.160897225990557), (4, 0.025250867217003526), (5, 0.028243355276995355), (6, 0.20741966185697386), (7, -0.02949998892685653), (8, 0.016054344303601472), (9, -0.06561837575565715)]
[(0, 0.13506079008785435), (1, 0.11689263323014258), (2, 0.03135893761009317), (3, 0.04783203004872515), (4, -0.1163914684726105), (5, 0.047199073578219036), (6, -0.04648518228498797),

Let's use these scores to fetch best-fit classifications for all of our (classifiable) patents:

In [53]:
classifications = [np.argmax(np.asarray(corpus_lsi[i])[:,1]) for i in range(len(stemmed_title_common_words_nonnull))]

In [54]:
topics = pd.DataFrame({'topic': classifications, 'title': classifiable_titles})

Certain topics that our classifier arrives at are much more common than others.

In [55]:
topics['topic'].value_counts()

0    260622
1     42014
3     24496
4     13912
9     10527
8      8873
6      8659
7       577
2       395
5       347
Name: topic, dtype: int64

## Visuals

Let's see what our classes look like.

In [56]:
from IPython.display import display

In [57]:
for i in range(10):
    print("Topic", i + 1)
    display(topics.query('topic == @i').head(5))

Topic 1


,topic,title
1,0,"[Method, Of, Treating, Intraoccular, Tissue, P..."
2,0,"[Method, And, Apparatus, For, Providing, An, E..."
3,0,"[Computer-Implemented, Method, ,, System, And,..."
4,0,"[Selecting, Graphical, Component, Types, At, R..."
5,0,"[Removal, Of, High, Molecular, Weight, Aggrega..."


Topic 2


,topic,title
0,1,"[Relevance, Sorting, For, Database, Searches]"
19,1,"[Implicit, Searching, For, Mobile, Content]"
53,1,"[Call, Control, Server]"
77,1,"[Receiver, For, A, Differential, Data, Bus]"
81,1,"[Data, Rate, Acquisition, Using, Signal, Edges]"


Topic 3


...

,topic,title
1465,7,"[Handling, Complex, Regex, Patterns, Storage-E..."
4077,7,"[Generic, Information, Element]"
4809,7,"[Buddy, Lists, For, Information, Vehicles]"
5255,7,"[Paper, Sheet, Handling, Apparatus]"
9900,7,"[Setting, User-Preference, Information, On, Th..."


Topic 9


,topic,title
580,8,"[Trench, Mos, Structure]"
618,8,"[Connector, For, Chip-Card]"
717,8,"[Multitrack, Optical, Disc, Reader]"
909,8,"[Planarising, Damascene, Structures]"
1213,8,"[Coil, Construction]"


Topic 10


,topic,title
26,9,"[Mobile, Search, Result, Clustering]"
27,9,"[Mobile, Search, Service, Discovery]"
30,9,"[Physical, Navigation, Of, A, Mobile, Search, ..."
41,9,"[Methods, ,, Systems, ,, And, Computer, Progra..."
88,9,"[Wireless, Terminal, ,, Wireless, Module, And,..."
